In [192]:
import pandas as pd
import numpy as np

def load_config():      
    param = np.genfromtxt("config.csv",delimiter=',',dtype=None)    
    par=[]    
    par.append(np.int16(param[0])) # Number of nodes
    par.append(np.int16(param[1])) # Max. Iterations
    par.append(np.float(param[2])) # Learn rate    
    return (par)
    
# Load data 
def load_data(fname):
    df = pd.read_csv(fname, header=None)   
    df = df.sample(frac=1).reset_index(drop=True) # Reordenar valores aleatoriamente.

    x, y = norm(df) # Normalizar

    return (x, y)

def save_w(w1,w2, fname_w, cost, fname_cost):    
    # Guardar pesos 
    np.savez_compressed(fname_w, w1, w2)
    # Guardar mse
    archivo = open(fname_cost, 'w')
    archivo.write('Iteracion,MSE\n') # header
    [ archivo.write(f'{i},{c}\n') for i, c in enumerate(cost, 1) ]
    archivo.close()

def load_w(fname):
    #completar code
    w1 = np.load(fname)['arr_0']
    w2 = np.load(fname)['arr_1']

    return (w1,w2)   

def norm(df):
    a = 0.01
    b = 0.99
    df_norm = pd.DataFrame()

    for col in df:
        min_ = df[col].min()
        max_ = df[col].max()

        df_norm[col] = (df[col] - min_) * (b - a) / (max_ - min_) + a
    
    x = df_norm.drop(labels=[5], axis=1)
    x = x.T
    y = df_norm.drop(labels=[i for i in range(5)],axis = 1)
    
    return (x.to_numpy(),y.to_numpy())

In [193]:
def iniW(nodes_hidden, x, y):
    size_input, _ = x.shape # 375, 5
    _, size_output = y.shape # 375, 1
    
    # Init. w1 y w2 
    w1 = np.random.random((nodes_hidden, size_input)) #dim -> (20x5)
    w2 = np.random.random((size_output, nodes_hidden)) #dim -> (1x20)
       
    return(w1,w2)

def grad_bp(Act, w1, w2, e):
    '''
    Dimensiones a considerar 
    e = (1, 375)
    w1 = (20, 5) | w2 = (1, 20)
    delta1 = (20, 5) | delta2 = (1, 375)
    dCdW1 = (20, 5)  | dCdW2 = (1, 20)
    '''
    a2 = Act                # 1, 375
    z2 = deriva_sigmoid(a2) # 1, 375
    a1 = np.dot(w2.T, z2)   # 20, 375
    z1 = deriva_sigmoid(a1) # 20, 375
    x = np.dot(w1.T, z1)    # 5, 375

    # Calcular gradiente capa salida
    delta2 = np.multiply(e, deriva_sigmoid(z2))
    dCdW2 = np.dot(delta2, a1.T)
    # Calcular gradiente capa oculta
    delta1 = np.multiply( np.dot(w2.T, delta2), deriva_sigmoid(z1) )
    dCdW1 = np.dot( delta1, x.T)
    
    return dCdW1, dCdW2

# Update SNN's Weight 
def updW(w1, w2, mu=0.1, dCdW=(0,0)):    
    dCdW1, dCdW2 = dCdW
    # Actualizar pesos ocultos
    w1 = w1 - mu * dCdW1   
    # Actualizar pesos salida
    w2 = w2 - mu * dCdW2
    return w1, w2

#Activation function
def act_sigmoid(z):
    return(1/(1+np.exp(-z)))   

# Derivate of the activation funciton
def deriva_sigmoid(a):
    return(a*(1-a))

In [194]:
def forward(x,w1,w2):
    # Calcula la activación de los Nodos Ocultos
    z1 = np.dot(w1, x)
    a1 = act_sigmoid(z1)
    # Calcula la activación de los Nodos de Salida
    z2 = np.dot(w2, a1)
    a2 = act_sigmoid(z2)

    return a2
    
def backward(Act, y, w1,w2, mu):
    # Calcular el error
    error = y.T - Act
    # Calcular el gradiente oculto y salida    
    dCdW = grad_bp(Act, w1, w2, error)
    # Actualizar los pesos
    w1, w2 = updW(w1, w2, mu, dCdW)
    
    mse = 0
    for e in error[0]:
        mse += e**2 * 0.5
    
    return w1, w2, mse 

In [196]:
#Training of SNN
def train_snn(x,y,param):
    w1,w2 = iniW(param[0], x, y)  
    cost = []
    for iter in range(param[1]):
        #Step 1: Forward
        Act = forward(x,w1,w2) 
      
        #Step2: Backward
        w1, w2, mse = backward(Act, y, w1,w2,param[2]) 
        cost.append(mse)
        
    return(w1,w2,cost)
    
# Beginning ...
def main():
    par_snn         = load_config()    
    xe,ye           = load_data('train.csv')     
    w1,w2, cost     = train_snn(xe,ye,par_snn)         
    save_w(w1,w2,'w_snn.npz',cost,'costo.csv')

main()

<ipython-input-192-4a6d1e6c4827>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  par.append(np.float(param[2])) # Learn rate
